## Load and prepare data

In [1]:
using DelimitedFiles

In [2]:
raw_data=readdlm("bingoBoards.txt");

In [85]:
drawnNumbers = raw_data[1]

"46,79,77,45,57,34,44,13,32,88,86,82,91,97,89,1,48,31,18,10,55,74,24,11,80,78,28,37,47,17,21,61,26,85,99,96,23,70,3,54,5,41,50,63,14,64,42,36,95,52,76,68,29,9,98,35,84,83,71,49,73,58,56,66,92,30,51,20,81,69,65,15,6,16,39,43,67,7,59,40,60,4,90,72,22,0,93,94,38,53,87,27,12,2,25,19,8,62,33,75"

In [86]:
drawnNumbers = parse.(Int,split(drawnNumbers,','));

In [5]:
bingoBoards = convert.(Int,raw_data[2:end,:]);

In [6]:
bingoBoards = permutedims(reshape(transpose(bingoBoards),(5,5,100)),(3,2,1))
bingoBoards[1,:,:]

5×5 Array{Int64,2}:
 84  94  24  52  44
 96  33  74  35  13
 60  51  41  19  95
 50  93  27  40   1
 67  23  37  88  85

### Problem test data

In [52]:
drawnNumbersTest=[7; 4; 9; 5; 11; 17; 23; 2; 0; 14; 21; 24; 10; 16; 13; 6; 15; 25; 12; 22; 18; 20; 8; 19; 3; 26; 1;]

bingoBoardsTest =[22 13 17 11  0
     8  2 23  4 24
    21  9 14 16  7
     6 10  3 18  5
     1 12 20 15 19
     3 15  0  2 22
     9 18 13 17  5
    19  8  7 25 23
    20 11 10 24  4
    14 21 16 12  6
    14 21 17 24  4
    10 16 15  9 19
    18  8 23 26 20
    22 11 13  6  5
    2  0 12  3  7]

15×5 Array{Int64,2}:
 22  13  17  11   0
  8   2  23   4  24
 21   9  14  16   7
  6  10   3  18   5
  1  12  20  15  19
  3  15   0   2  22
  9  18  13  17   5
 19   8   7  25  23
 20  11  10  24   4
 14  21  16  12   6
 14  21  17  24   4
 10  16  15   9  19
 18   8  23  26  20
 22  11  13   6   5
  2   0  12   3   7

In [53]:
bingoBoardsTest = permutedims(reshape(transpose(bingoBoardsTest),(5,5,3)),(3,2,1))
bingoBoardsTest[1,:,:]

5×5 Array{Int64,2}:
 22  13  17  11   0
  8   2  23   4  24
 21   9  14  16   7
  6  10   3  18   5
  1  12  20  15  19

## Part 1

In [7]:
println(length(bingoBoards))
size(bingoBoards)

2500


(100, 5, 5)

In [16]:
typeof(size(drawn_numbers))

Tuple{Int64}

In [8]:
nboards=size(bingoBoards)[1]
boardSize=size(bingoBoards)[2]

5

In [82]:
function evaluateBoards(boards::Array{Int,3},numbers::Array{Int,1})
    
    nboards=size(boards)[1]
    boardSize=size(boards)[2]
    masks = convert.(Int,zeros(size(boards)));
    
    for i in 1:length(numbers)
        # find all board entries corresponding to number
        for j in 1:length(boards)
            if boards[j] == numbers[i]
                masks[j] = 1
            end
        end
        
        # check if one of the boards has reached bingo (5 ones in row/column)
        for j in 1:nboards,k in 1:boardSize
            rowSum=sum(masks[j,:,k])
            colSum=sum(masks[j,k,:])
            if (rowSum==5) || (colSum==5)
                #on Bingo, exit and return number called, the board that has bingo, and the bingo mask
                println("Bingo in board ",j)
                return numbers[i], boards[j,:,:], masks[j,:,:] 
            end
        end
    end
end

evaluateBoards (generic function with 1 method)

In [83]:
result=evaluateBoards(bingoBoardsTest,drawnNumbersTest)

Bingo in board 3


(24, [14 21 … 24 4; 10 16 … 9 19; … ; 22 11 … 6 5; 2 0 … 3 7], [1 1 … 1 1; 0 0 … 1 0; … ; 0 1 … 0 1; 1 1 … 0 1])

In [84]:
println(result)

(24, [14 21 17 24 4; 10 16 15 9 19; 18 8 23 26 20; 22 11 13 6 5; 2 0 12 3 7], [1 1 1 1 1; 0 0 0 1 0; 0 0 1 0 0; 0 1 0 0 1; 1 1 0 0 1])


In [88]:
winNumber,winBoard,winMask=evaluateBoards(bingoBoards,drawnNumbers)

Bingo in board 61


(28, [19 85 … 73 71; 65 62 … 52 3; … ; 55 15 … 61 95; 28 13 … 31 88], [0 0 … 0 0; 0 0 … 0 0; … ; 1 0 … 0 0; 1 1 … 1 1])

In [102]:
score=sum(winBoard[winMask.==0])
println(score,' ',score*winNumber)

810 22680


### Testing julia loops and break statements

In [54]:
using Random

In [72]:
dummy = bitrand(10,5,5);

In [77]:
for i in 1:10,j in 1:boardSize
    rowSum=sum(dummy[i,:,j])
    colSum=sum(dummy[i,j,:])
    if (rowSum==5) || (colSum==5)
        println("Bingo: ",i)
        break
    end
end

Bingo: 2


In [80]:
dummy[2,:,:]

5×5 BitArray{2}:
 0  0  1  0  0
 0  1  1  1  0
 0  0  1  1  1
 1  0  1  0  0
 0  1  1  1  0

## Part 2

In [120]:
function evaluateBoardsLast(boards::Array{Int,3},numbers::Array{Int,1})
    
    nboards=size(boards)[1]
    boardSize=size(boards)[2]
    masks = zeros(Int,size(boards))
    hasBingo = zeros(Int,nboards)
    bingoWhen = zeros(Int,nboards)
    
    for i in 1:length(numbers)
        # find all board entries corresponding to number
        for j in 1:length(boards)
            if boards[j] == numbers[i]
                masks[j] = 1
            end
        end
        
        # check if one of the boards has reached bingo (5 ones in row/column)
        for j in 1:nboards
            #check if board has already had bingo so we can skip it
            if hasBingo[j] == 1
                continue     
            end
            for k in 1:boardSize
                rowSum=sum(masks[j,:,k])
                colSum=sum(masks[j,k,:])
                if (rowSum==5) || (colSum==5)
                    #on Bingo, mark bingo in hasBingo and save index of number to bingoWhen
                    hasBingo[j] = 1
                    bingoWhen[j] = i
                end
            end
        end
        
        #check if any board has not yet reached bingo, leave otherwise
        if count(x->x==0,hasBingo)==0
            break
        end
    end
    
    #return the board with the highest number in bingoWhen and the corresponding number, board number and mask
    numberIndexLast,boardIndexLast = findmax(bingoWhen)
    return numbers[numberIndexLast], boardIndexLast, boards[boardIndexLast,:,:],masks[boardIndexLast,:,:]
end

evaluateBoardsLast (generic function with 1 method)

In [121]:
resultsTest = evaluateBoardsLast(bingoBoardsTest,drawnNumbersTest)

(13, 2, [3 15 … 2 22; 9 18 … 17 5; … ; 20 11 … 24 4; 14 21 … 12 6], [0 0 … 1 0; 1 0 … 1 1; … ; 0 1 … 1 1; 1 1 … 0 0])

In [122]:
resultsTest[4]

5×5 Array{Int64,2}:
 0  0  1  1  0
 1  0  1  1  1
 0  0  1  0  1
 0  1  1  1  1
 1  1  1  0  0

In [123]:
results = evaluateBoardsLast(bingoBoards,drawnNumbers)

(94, 66, [72 49 … 12 43; 73 56 … 4 27; … ; 14 96 … 8 13; 87 60 … 77 42], [1 1 … 0 1; 1 1 … 1 0; … ; 1 1 … 0 1; 0 1 … 1 1])

In [124]:
looseBoard = results[3]
looseMask = results[4]
looseNumber = results[1]
score=sum(looseBoard[looseMask.==0])
println(score,' ',score*looseNumber)

172 16168
